In [1]:
import torchaudio
from hubert_kmeans import HubertWithKmeans
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

inference_pipeline = pipeline(
    task=Tasks.voice_activity_detection,
    model='damo/speech_fsmn_vad_zh-cn-16k-common-pytorch',
    model_revision=None,
)

2023-09-26 01:22:14 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-09-26 01:22:15,693 - modelscope - INFO - PyTorch version 2.0.1 Found.
2023-09-26 01:22:15,695 - modelscope - INFO - Loading ast index from /home/shun.lei/.cache/modelscope/ast_indexer
2023-09-26 01:22:15,740 - modelscope - INFO - Loading done! Current index file version is 1.9.1, with md5 587f978a81588255a7cec59dbf338d0b and a total number of 924 components indexed
2023-09-26 01:22:18,135 - modelscope - INFO - Model revision not specified, use the latest revision: v1.2.0
2023-09-26 01:22:18,475 - modelscope - INFO - initiate model from /home/shun.lei/.cache/modelscope/hub/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch
2023-09-26 01:22:18,475 - modelscope - INFO - initiate model from location /home/shun.lei/.cache/modelscope/hub/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.
2023-09-26 01:22:18,477 - modelscope - INFO - initialize model from /home/shun.lei/.cache/modelsc

In [2]:
from pathlib import Path

# 先获取语音的数据
folder_list = ['/data4/nas40T/share/shun.lei/workspace/data/raw_data/LibriTTS/dev-clean','/data4/nas40T/share/shun.lei/workspace/data/raw_data/LibriTTS/dev-other','/data4/nas40T/share/shun.lei/workspace/data/raw_data/LibriTTS/test-clean','/data4/nas40T/share/shun.lei/workspace/data/raw_data/LibriTTS/test-other','/data4/nas40T/share/shun.lei/workspace/data/raw_data/LibriTTS/train-clean-100','/data4/nas40T/share/shun.lei/workspace/data/raw_data/LibriTTS/train-clean-360','/data4/nas40T/share/shun.lei/workspace/data/raw_data/LibriTTS/train-other-500']
suffix = '.wav'
speech_list = []
for folder in folder_list:
    folder = Path(folder)
    path = [*folder.rglob(f'*{suffix}')]
    speech_list.extend(path)

print(len(speech_list))


375085


In [5]:
from tqdm import tqdm
import os
import sys

speech_rate = []
for data in tqdm(speech_list):
    wav_path = data
    sys.stdout = open(os.devnull, 'w')
    result = inference_pipeline(audio_in=str(wav_path))
    sys.stdout = sys.__stdout__
    if 'text' not in result:
        print(data)
    result = result['text']
    start = result[0][0]
    end = result[-1][-1]
    duration = (end-start)
    cnt = 0
    for time in result:
        cnt += time[1]-time[0]
    rate = cnt/duration
    speech_rate.append(rate)


  0%|                                                                                                                                                                                                                                                                                                | 1/375085 [00:04<479:46:12,  4.60s/it]2023-09-26 01:28:27,029 - modelscope - INFO - VAD Processing ...
2023-09-26 01:28:27,041 - modelscope - INFO - VAD Processing ...
  0%|                                                                                                                                                                                                                                                                                                 | 9/375085 [00:05<35:39:40,  2.92it/s]2023-09-26 01:28:28,392 - modelscope - INFO - VAD Processing ...
2023-09-26 01:28:28,482 - modelscope - INFO - VAD Processing ...
  0%|                                                                       

KeyboardInterrupt: 

In [10]:
import json

# 存放解析后的 JSON 数据
data_list = []

with open('/data4/leishun/text_to_semantic/datasets/new_segment.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        data_list.append(data)

# 打印读取到的数据
for item in data_list[:15]:
    print(item)

{'utt': 'Xhr3e5y_fvw-0', 'start': 3.48, 'end': 7.33, 'text': '[SEP] ♪ OOH MMM ♪', 'wav': '/data4/music-corpus/yt-best/separated_opus/htdemucs/Xhr3e5y_fvw/vocals.opus', 'dur': 3.85}
{'utt': 'Xhr3e5y_fvw-1', 'start': 10.61, 'end': 22.69, 'text': "♪ AY AY ♪ WAKE UP TO THE SUNLIGHT ♪ CREEPING THROUGH THE WINDOW ♪ GLOWING ON YOUR BODY ♪ CAN WE KEEP IT SIMPLE ♪ I NEED MORE THAN ONE NIGHT ♪ WHAT YOU THINK I'M HERE FOR ♪", 'wav': '/data4/music-corpus/yt-best/separated_opus/htdemucs/Xhr3e5y_fvw/vocals.opus', 'dur': 12.080000000000002}
{'utt': 'Xhr3e5y_fvw-2', 'start': 22.69, 'end': 37.67, 'text': "♪ YOU KNOW THAT YOU GOT ME ♪ BEAT IS TO THE TEMPO ♪ CAN WE CUT RIGHT TO THE CHASE (CHASE) ♪ I'M SOMEONE WITH COMPLICATED TASTE ♪ [SEP] ♪ YOU CAN FINISH FIRST THIS AIN'T A RACE ♪ [SEP] ♪ IMMA LOCK IT UP ♪ GIRL LEAD THE WAY ♪", 'wav': '/data4/music-corpus/yt-best/separated_opus/htdemucs/Xhr3e5y_fvw/vocals.opus', 'dur': 14.98}
{'utt': 'Xhr3e5y_fvw-3', 'start': 37.67, 'end': 50.9, 'text': "♪ YOU KNOW WHAT

In [18]:
from tqdm import tqdm
import os
import sys

c = 0
for data in tqdm(data_list[:100]):
    wav_path = os.path.join('/data4/leishun/processed_data/raw_cut_vocal_16k',data['utt']+'.wav')
    dur = data['dur']
    sys.stdout = open(os.devnull, 'w')
    result = inference_pipeline(audio_in=wav_path)
    sys.stdout = sys.__stdout__
    if 'text' not in result:
        print(data)
        continue
    result = result['text']
    start = result[0][0]
    end = result[-1][-1]
    duration = (end-start)
    cnt = 0
    for time in result:
        cnt += time[1]-time[0]
        
    rate = cnt/duration
    if rate < 0.9:
        c+=1
print(c)

 44%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                    | 44/100 [00:16<00:19,  2.81it/s]2023-09-26 01:17:31,766 - modelscope - INFO - VAD Processing ...
2023-09-26 01:17:31,843 - modelscope - INFO - VAD Processing ...
 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                          | 91/100 [00:30<00:03,  2.66it/s]2023-09-26 01:17:46,636 - modelscope - INFO - VAD Processing ...
2023-09-26 01:17:46,697 - modelscope - INFO - VAD Processing ...
100%|███████████████████████████████████████████████████████████████████████

In [19]:
c

26

In [2]:
from tqdm import tqdm
# 按music名字分类
music_data = {}

for data in data_list:
    name = '-'.join(data['utt'].split('-')[:-1])
    if name not in music_data:
        music_data[name] = []
    music_data[name].append(data)


In [3]:
from pathlib import Path
import os
for k in tqdm(music_data.keys()):
    music = music_data[k]
    wav_path = music[0]['wav']
    wav_path = Path(wav_path)
    if not wav_path.exists():
        print(wav_path)

 31%|███████████▍                         | 7908/25714 [00:12<00:28, 627.10it/s]

/data4/music-corpus/yt-best/separated_opus/htdemucs/vEM6ry3s5dI/vocals.opus


 44%|███████████████▋                    | 11247/25714 [00:17<00:26, 549.93it/s]

/data4/music-corpus/yt-best/separated_opus/htdemucs/vER5NAlbQHw/vocals.opus


100%|████████████████████████████████████| 25714/25714 [00:26<00:00, 972.82it/s]
